学习推荐：https://github.com/TingsongYu/PyTorch-Tutorial-2nd
# 多显卡使用
```
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "2,1,3,4"
```
2号卡就变成了主卡。CUDA_VISIBLE_DEVICES 表示当前可以被python环境程序检测到的显卡。os.environ["CUDA_VISIBLE_DEVICES"] = "2,1,3,4"进行指定使用设备，这样会修改pytorch感受的设备编号，pytorch感知的编号还是从device:0开始。如上会把2号显卡改为device:0，1号显卡改为device:1。

查看显卡使用情况：cmd:nvidia-smi
# 显卡占用
cmd:
查询占用情况：nvidia-smi
关闭PID为“xxxx”的进程：taskkill -PID xxxx -F
再次查询占用情况：nvidia-smi
# 基本使用
- 定义和编辑
```
# 定义张量Tensors：pytorch 的一大作用就是可以代替 Numpy 库，相当于 Numpy 的多维数组(ndarrays)。两者的区别就是 Tensors 可以应用到 GPU 上加快计算速度。

x = torch.empty(5, 3) # 创建一个 5*3 空矩阵
rand_x = torch.rand(5, 3) # 创建一个随机初始化的 5*3 矩阵
zero_x = torch.zeros(5, 3, dtype=torch.long) # 创建一个数值皆是 0，类型为 long 的矩阵。类似的也可以创建数值都是 1 的矩阵，调用 torch.ones。
tensor1 = torch.tensor([5.5, 3]) # 传递数值创建张量
tensor2 = tensor1.new_ones(5, 3, dtype=torch.double)  # 根据已有的 tensor 变量创建新的 tensor 变量，这种做法的好处就是可以保留已有 tensor 的一些属性，包括尺寸大小、数值属性，除非是重新定义这些属性。new_* 方法需要输入 tensor 大小。
tensor2.size() # tensors 的尺寸大小获取

# 改变 tensor 变量的操作都带有一个后缀 _, 例如 x.copy_(y), x.t_() 都可以改变 x 变量
# 对于 Tensor 的访问，和 Numpy 对数组类似，可以使用索引来访问某一维的数据
# 对 Tensor 的尺寸修改，可以采用 torch.view()
# 和 Numpy 数组的转换：Tensor 和 Numpy 的数组可以相互转换，并且两者转换后共享在 CPU 下的内存空间，即改变其中一个的数值，另一个变量也会随之改变。在 CPU 上，除了 CharTensor 外的所有 Tensor 类型变量，都支持和 Numpy数组的相互转换操作。
# Tensor 转换为 Numpy 数组：tensor1.numpy()
# Numpy 数组转换为 Tensor：torch.from_numpy(numpy_array)

# Tensors 可以通过 .to 方法转换到不同的设备上，即 CPU 或者 GPU 上。
if torch.cuda.is_available():
    x = torch.rand(5, 3)
    device = torch.device("cuda")          # 定义一个 CUDA 设备对象
    y = torch.ones_like(x, device=device)  # 显示创建在 GPU 上的一个 tensor
    x = x.to(device)                       # 也可以采用 .to("cuda") 
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))       # .to() 方法也可以改变数值类型
```
- autograd：autograd主要是提供了对 Tensors 上所有运算操作的自动微分功能，也就是计算梯度的功能。它属于 define-by-run 类型框架，即反向传播操作的定义是根据代码的运行方式，因此每次迭代都可以是不同的。
    - torch.Tensor
        - 设置属性.requires_grad=True，那么就会开始追踪在该变量上的所有操作；而完成计算后，可以调用 .backward() 并自动计算所有的梯度，得到的梯度都保存在属性 .grad 中。
        - 调用 .detach() 方法分离出计算的历史，可以停止一个 tensor 变量继续追踪其历史信息 ，同时也防止未来的计算会被追踪。
        - 而如果是希望防止跟踪历史（以及使用内存），可以将代码块放在 with torch.no_grad(): 内，这个做法在使用一个模型进行评估的时候非常有用，因为模型会包含一些带有 requires_grad=True 的训练参数，但实际上并不需要它们的梯度信息。
    - Function：Tensor 和 Function 两个类是有关联并建立了一个非循环的图，可以编码一个完整的计算记录。每个 tensor 变量都带有属性 .grad_fn ，该属性引用了创建了这个变量的 Function （除了由用户创建的 Tensors，它们的 grad_fn=None） 。
        - 梯度d(Tensor_y)/dx获取：Tensor_y.backward() ；res=x.grad
- torch.nn：专门用于实现神经网络
    - 定义一个多层的神经网络：
    - 对数据集的预处理并准备作为网络的输入
    - 将数据输入到网络
    - 计算网络的损失
    - 反向传播，计算梯度
    - 更新网络的梯度，一个简单的更新规则是 weight = weight - learning_rate * gradient


```

```

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [2]:
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(torch.cuda.is_available()) # 验证能否正确运行在 GPU 上

ng=torch.cuda.device_count() # 查看当前设备环境可用显卡数量
infos=[torch.cuda.get_device_properties(i) for i in range(ng)] # 查看可用显卡的具体信息（型号、算力，显存以及线程数）
print(ng)
print(infos)

1.13.0+cu117
11.7
8500
False
0
[]


In [7]:
# 定义一个神经网络：下面是一个 5 层的卷积神经网络，包含两层卷积层和三层全连接层：
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 输入图像是单通道，conv1 kenrnel size=5*5，输出通道 6
        self.conv1 = nn.Conv2d(1, 6, 5)
        # conv2 kernel size=5*5, 输出通道 16
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 全连接层
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # max-pooling 采用一个 (2,2) 的滑动窗口
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 核(kernel)大小是方形的话，可仅定义一个数字，如 (2,2) 用 2 即可
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        # 除了 batch 维度外的所有维度
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [ ]:
# 数据预处理：一般采用标准的 Python 库将其加载并转成 Numpy 数组，然后再转回为 PyTorch 的张量。
## 对于图像，可以采用 Pillow, OpenCV 库；
## 对于语音，有 scipy 和 librosa;
## 对于文本，可以选择原生 Python 或者 Cython 进行加载数据，或者使用 NLTK 和 SpaCy 。
## torchvision 库：包含一个数据加载器(data loader)，可以加载比较常见的数据集，比如 Imagenet, CIFAR10, MNIST 等等，然后还有一个用于图像的数据转换器(data transformers)，调用的库是 torchvision.datasets 和 torch.utils.data.DataLoader 。

# 通过调用 torchvision 加载和归一化 CIFAR10 训练集和测试集；





In [ ]:
# 构建一个卷积神经网络；
# 定义一个损失函数；
# 在训练集上训练网络；
# 在测试集上测试网络性能。